<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [126]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [127]:
import folium
import pandas as pd

In [128]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon



If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [129]:
## Task 1: Mark all launch sites on a map

# Step 1: Create a dictionary of known launch sites with coordinates
launch_sites = {
    'CCAFS LC-40': [28.5623, -80.5774],
    'KSC LC-39A': [28.6084, -80.6043],
    'VAFB SLC-4E': [34.6321, -120.6106],
    'CCAFS SLC-40': [28.5632, -80.5768]
}

# Step 2: Convert to a DataFrame for easy iteration
launch_sites_df = pd.DataFrame([
    {'Site': name, 'Lat': coord[0], 'Lon': coord[1]}
    for name, coord in launch_sites.items()
])

# Step 3: Create a folium map centered around Cape Canaveral
map_launch_sites = folium.Map(location=[28.5, -80.6], zoom_start=5)

# Step 4: Add a marker for each launch site
for _, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Lon']],
        popup=row['Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(map_launch_sites)

# Step 5: Display the map (will render in Jupyter or similar notebook)
map_launch_sites

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [130]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [131]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [132]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [171]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [172]:
# Initial the map
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Center map on Cape Canaveral
nasa_coordinate = [28.5623, -80.5774]

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Dictionary of launch sites
launch_sites = {
    'CCAFS LC-40': [28.5623, -80.5774],
    'KSC LC-39A': [28.6084, -80.6043],
    'VAFB SLC-4E': [34.6321, -120.6106],
    'CCAFS SLC-40': [28.5632, -80.5768]
}

# Add circles with popup labels
for site, coord in launch_sites.items():
    folium.Circle(
        location=coord,
        radius=500,             # radius in meters
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=folium.Popup(site, parse_html=True)
    ).add_to(site_map)


    # Add floating text label (DivIcon)
    folium.map.Marker(
        location=[coord[0], coord[1] + 0.05],  # slightly offset longitude to the right
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 8pt; color: red;"><b>{site}</b></div>'
        )
    ).add_to(site_map)

# Display the map (in Jupyter notebook)
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [173]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
from folium.plugins import MarkerCluster

# Step 1: Add a marker color column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 2: Initialize the map
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=6)

# Step 3: Add a marker cluster to group nearby launches
marker_cluster = MarkerCluster().add_to(site_map)

# Step 4: Loop through the dataframe and add markers
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color='white', icon_color=row['marker_color'], icon='info-sign'),
        popup=f"Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Step 5: Display the map
site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [174]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color,dist_to_railroad,dist_to_highway,dist_to_city,dist_to_coastline
46,KSC LC-39A,28.573255,-80.646895,1,green,6.016925,6.907310,71.631466,14.690106
47,KSC LC-39A,28.573255,-80.646895,1,green,6.016925,6.907310,71.631466,14.690106
48,KSC LC-39A,28.573255,-80.646895,1,green,6.016925,6.907310,71.631466,14.690106
49,CCAFS SLC-40,28.563197,-80.576820,1,green,1.356485,0.158273,78.422735,9.607205
50,CCAFS SLC-40,28.563197,-80.576820,1,green,1.356485,0.158273,78.422735,9.607205
51,CCAFS SLC-40,28.563197,-80.576820,0,red,1.356485,0.158273,78.422735,9.607205
52,CCAFS SLC-40,28.563197,-80.576820,0,red,1.356485,0.158273,78.422735,9.607205
53,CCAFS SLC-40,28.563197,-80.576820,0,red,1.356485,0.158273,78.422735,9.607205
54,CCAFS SLC-40,28.563197,-80.576820,1,green,1.356485,0.158273,78.422735,9.607205
55,CCAFS SLC-40,28.563197,-80.576820,0,red,1.356485,0.158273,78.422735,9.607205


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [175]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [176]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

import numpy as np

spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [177]:
# Add marker_cluster to current site_map
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered at a general location
site_map = folium.Map(location=[28.5, -80.5], zoom_start=6)

# Create a MarkerCluster
marker_cluster = MarkerCluster()

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Loop through each row in the DataFrame
for index, record in spacex_df.iterrows():
    # Create a marker with coordinates and custom icon color
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {record['class']}"
    )
    # Add marker to cluster
    marker_cluster.add_child(marker)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [178]:
# TASK 3: Calculate the distances between a launch site to its proximities
railroad_coords = (28.572872, -80.585281)  # Example rail station near Cape Canaveral
highway_coords = (28.561857, -80.577366)   # Example highway location
city_coords = (28.538336, -81.379234)      # Orlando city center
coastline_coords = (28.485, -80.535)       # Example coastline point

import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)

    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * \
        math.cos(math.radians(lat2)) * math.sin(dlon/2)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Calculate distances and add as new columns
spacex_df['dist_to_railroad'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], railroad_coords[0], railroad_coords[1]), axis=1)

spacex_df['dist_to_highway'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], highway_coords[0], highway_coords[1]), axis=1)

spacex_df['dist_to_city'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], city_coords[0], city_coords[1]), axis=1)

spacex_df['dist_to_coastline'] = spacex_df.apply(
    lambda row: haversine(row['Lat'], row['Long'], coastline_coords[0], coastline_coords[1]), axis=1)

spacex_df[['Launch Site', 'dist_to_railroad', 'dist_to_highway', 'dist_to_city', 'dist_to_coastline']]

,Launch Site,dist_to_railroad,dist_to_highway,dist_to_city,dist_to_coastline
0,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
1,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
2,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
3,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
4,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
5,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
6,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
7,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
8,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831
9,CCAFS LC-40,1.407235,0.049487,78.367256,9.539831


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [179]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [180]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [181]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# 👆 Coordinates you copied using MousePosition
coastline_lat = 28.5622
coastline_lon = -80.56781

# 👇 Get launch site coordinates
launch_site = spacex_df[spacex_df['Launch Site'] == spacex_df['Launch Site'].iloc[0]]
launch_lat = launch_site['Lat'].iloc[0]
launch_lon = launch_site['Long'].iloc[0]

# 💡 Calculate distance
distance_to_coast = calculate_distance(launch_lat, launch_lon, coastline_lat, coastline_lon)
print(f"Distance from launch site to coastline: {distance_to_coast:.2f} km")

Distance from launch site to coastline: 0.93 km


In [183]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from math import radians, sin, cos, sqrt, atan2

# --- Haversine distance ---
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# --- Coordinates ---
coastline_coords = (28.5622, -80.56781)

# Get a launch site (first in the dataset)
launch_lat = spacex_df['Lat'].iloc[0]
launch_lon = spacex_df['Long'].iloc[0]
launch_coords = (launch_lat, launch_lon)

# --- Calculate distance ---
distance_km = calculate_distance(launch_lat, launch_lon, *coastline_coords)

# --- Create map ---
site_map = folium.Map(location=launch_coords, zoom_start=12)

# --- Add launch site marker ---
folium.Marker(
    launch_coords,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='rocket')
).add_to(site_map)

# --- Add coastline marker ---
folium.Marker(
    coastline_coords,
    popup="Closest Coastline",
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(site_map)

# --- Add distance label ---
folium.Marker(
    [(launch_lat + coastline_coords[0]) / 2, (launch_lon + coastline_coords[1]) / 2],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Show the map ---
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [184]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
import folium

# Coordinates (replace with your actual values)
launch_coords = [28.562302, -80.577356]       # Example launch site
coastline_coords = [28.5622, -80.56781]    # Example coastline point

# 1. Create a list of points for the PolyLine
coordinates = [launch_coords, coastline_coords]

# 2. Create the map
site_map = folium.Map(location=launch_coords, zoom_start=13)

# 3. Add markers
folium.Marker(launch_coords, popup="Launch Site", icon=folium.Icon(color='blue')).add_to(site_map)
folium.Marker(coastline_coords, popup="Coastline", icon=folium.Icon(color='green')).add_to(site_map)

# 4. Create and add the PolyLine
lines = folium.PolyLine(locations=coordinates, weight=2, color='purple')
lines.add_to(site_map)

# 5. Display map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [195]:
import folium
import math

# Coordinates
launch_site = [28.5623, -80.5774]        # CCAFS LC-40
nearby_city = [28.38657, -80.60429]      # Cocoa, FL
railway = [28.5629, -80.58699]           # Nearby railway
highway = [28.56198, -80.57047]          # Nearby highway
coastline = [28.5622, -80.56781]         # Nearest coastline

# Haversine distance function (returns km)
def haversine(coord1, coord2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1 = map(math.radians, coord1)
    lat2, lon2 = map(math.radians, coord2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Initialize folium map
proximity_map = folium.Map(location=launch_site, zoom_start=13)

# Add launch site marker
folium.Marker(
    location=launch_site,
    popup='Launch Site: CCAFS LC-40',
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(proximity_map)

# Function to add markers and lines
def add_distance_marker(map_obj, location, label, color):
    distance_km = haversine(launch_site, location)
    distance_str = f"{distance_km:.2f} km"

    # Add marker
    folium.Marker(
        location=location,
        popup=f"{label}<br>Distance: {distance_str}",
        icon=folium.Icon(color=color)
    ).add_to(map_obj)

    # Draw line
    folium.PolyLine(
        [launch_site, location],
        color=color,
        weight=2,
        opacity=0.6
    ).add_to(map_obj)

# Add all nearby features
add_distance_marker(proximity_map, nearby_city, 'Nearest City: Cocoa, FL', 'blue')
add_distance_marker(proximity_map, railway, 'Nearest Railway', 'purple')
add_distance_marker(proximity_map, highway, 'Nearest Highway', 'green')
add_distance_marker(proximity_map, coastline, 'Nearest Coastline', 'orange')

# Display the map
proximity_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
